In [1]:
import pandas as pd
import numpy as np
import pyarrow
from tqdm import tqdm
import pyspark.sql.functions as F
from pyspark.sql.functions import pandas_udf, struct, PandasUDFType
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType, FloatType


#spark.conf.set("spark.sql.execution.arrow.enabled", 'true')

In [66]:
df1 = spark.read.parquet("/home/vinny/Documents/path_teste")

In [67]:
df1.printSchema()

root
 |-- path: string (nullable = true)
 |-- start: double (nullable = true)
 |-- end: double (nullable = true)
 |-- transcript: string (nullable = true)
 |-- speaker: string (nullable = true)



In [68]:
schema = StructType([
    StructField('filename', StringType(), True),
    StructField('start', FloatType(), True),
    StructField('end', FloatType(), True),
    StructField('transcript', StringType(), True)
])

In [107]:
base = df1

In [108]:
base = base.withColumnRenamed("path", "filename")

In [109]:
base.printSchema()

root
 |-- filename: string (nullable = true)
 |-- start: double (nullable = true)
 |-- end: double (nullable = true)
 |-- transcript: string (nullable = true)
 |-- speaker: string (nullable = true)



In [110]:
udf_capitalize = F.udf(lambda x: str(x).capitalize(), StringType())

In [111]:
base = base.withColumn("transcript", udf_capitalize("transcript"))

In [112]:
newDataframe = base.groupBy('filename').agg(F.concat_ws('. ', F.collect_list(base.transcript)).alias('transcript'))

In [113]:
newDataframe.show()

+----------------+--------------------+
|        filename|          transcript|
+----------------+--------------------+
|A23CDO549BFRGH54|Bom dia meu nome ...|
|A23CDO549BFRGH56|Bom dia meu nome ...|
|A23CDO549BFRGH57|Bom dia meu nome ...|
|A23CDO549BFRGH55|Bom dia meu nome ...|
+----------------+--------------------+



In [114]:
#cliente
#dtcliente = dataframe[dataframe['speaker']=='cliente'].groupby([coluna_grouped], as_index=False)[coluna_transc].agg({coluna_transc: '. '.join})
dtcliente = base.filter(base['speaker']=='cliente').groupBy('filename').agg(F.concat_ws('. ', F.collect_list(base.transcript)).alias('transcript_cliente'))
    
#atendente
#dtatendente = dataframe[dataframe['speaker']=='atendente'].groupby([coluna_grouped], as_index=False)[coluna_transc].agg({coluna_transc: '. '.join})
dtatendente = base.filter(base['speaker']=='atendente').groupBy('filename').agg(F.concat_ws('. ', F.collect_list(base.transcript)).alias('transcript_atendente'))


In [115]:
dtcliente =  dtcliente.withColumnRenamed('filename', 'filename_c')
dtatendente = dtatendente.withColumnRenamed('filename', 'filename_a')

In [116]:
dtcliente.show()

+----------------+--------------------+
|      filename_c|  transcript_cliente|
+----------------+--------------------+
|A23CDO549BFRGH54|Bom dia meu nome ...|
|A23CDO549BFRGH56|Bom dia meu nome ...|
|A23CDO549BFRGH57|Bom dia meu nome ...|
|A23CDO549BFRGH55|Bom dia meu nome ...|
+----------------+--------------------+



In [102]:
dtatendente.show()

+----------------+--------------------+
|      filename_a|transcript_atendente|
+----------------+--------------------+
|A23CDO549BFRGH54|Bom dia senhor ca...|
|A23CDO549BFRGH56|Bom dia senhor ca...|
|A23CDO549BFRGH57|Bom dia senhor ca...|
|A23CDO549BFRGH55|Bom dia senhor ca...|
+----------------+--------------------+



In [117]:
ls_cliente = newDataframe.join(dtcliente, newDataframe['filename']==dtcliente['filename_c'], 'inner')
ls_atendente = newDataframe.join(dtatendente, newDataframe['filename']==dtatendente['filename_a'], 'inner')
ls_atendente =  ls_atendente.withColumnRenamed('filename', 'filename_ls_a').withColumnRenamed('transcript', 'transcript_ls_a')
newDataframe2 = ls_cliente.join(ls_atendente, ls_cliente['filename']==ls_atendente['filename_ls_a'], 'inner').select('filename', 'transcript', 'transcript_cliente', 'transcript_atendente')

In [82]:
ls_cliente.show()

+----------------+--------------------+----------------+--------------------+
|        filename|          transcript|      filename_c|  transcript_cliente|
+----------------+--------------------+----------------+--------------------+
|A23CDO549BFRGH54|Bom dia meu nome ...|A23CDO549BFRGH54|Bom dia meu nome ...|
|A23CDO549BFRGH56|Bom dia meu nome ...|A23CDO549BFRGH56|Bom dia meu nome ...|
|A23CDO549BFRGH57|Bom dia meu nome ...|A23CDO549BFRGH57|Bom dia meu nome ...|
|A23CDO549BFRGH55|Bom dia meu nome ...|A23CDO549BFRGH55|Bom dia meu nome ...|
+----------------+--------------------+----------------+--------------------+



In [83]:
ls_atendente.show()

+----------------+--------------------+----------------+--------------------+
|        filename|          transcript|      filename_a|transcript_atendente|
+----------------+--------------------+----------------+--------------------+
|A23CDO549BFRGH54|Bom dia meu nome ...|A23CDO549BFRGH54|Bom dia senhor ca...|
|A23CDO549BFRGH56|Bom dia meu nome ...|A23CDO549BFRGH56|Bom dia senhor ca...|
|A23CDO549BFRGH57|Bom dia meu nome ...|A23CDO549BFRGH57|Bom dia senhor ca...|
|A23CDO549BFRGH55|Bom dia meu nome ...|A23CDO549BFRGH55|Bom dia senhor ca...|
+----------------+--------------------+----------------+--------------------+



In [124]:
newDataframe2.show()

+----------------+--------------------+--------------------+--------------------+
|        filename|          transcript|  transcript_cliente|transcript_atendente|
+----------------+--------------------+--------------------+--------------------+
|A23CDO549BFRGH54|Bom dia meu nome ...|Bom dia meu nome ...|Bom dia senhor ca...|
|A23CDO549BFRGH55|Bom dia meu nome ...|Bom dia meu nome ...|Bom dia senhor ca...|
|A23CDO549BFRGH56|Bom dia meu nome ...|Bom dia meu nome ...|Bom dia senhor ca...|
|A23CDO549BFRGH57|Bom dia meu nome ...|Bom dia meu nome ...|Bom dia senhor ca...|
+----------------+--------------------+--------------------+--------------------+



# exemplos

In [ ]:
def separar_tudo(newDataframe):
    ls_cliente = []
    ls_atendente = []
    for i in tqdm(newDataframe['filename'].unique(), total=len(newDataframe['filename'].unique())):
    #existe transcricao cliente
        dtcliente[dtcliente['filename']==i]
        if len(dtcliente[dtcliente['filename']==i]['transcript'])>0:
            ls_cliente.append(list(dtcliente[dtcliente['filename']==i]['transcript'])[0])
        else:
            ls_cliente.append('')

        #existe transcricao atendente
        dtatendente[dtatendente['filename']==i]
        if len(dtatendente[dtatendente['filename']==i]['transcript')>0:
            ls_atendente.append(list(dtatendente[dtatendente['filename']==i]['transcript'])[0])
        else:
            ls_atendente.append('')

    newDataframe['transcript_cliente'] = ls_cliente
    newDataframe['transcript_atendente'] = ls_atendente

#### EXEMPLO DE CRIACAO DE FUNCAO - GK Tech - UDF, convers'ao de tipos ...

In [4]:
# EXEMPLO DO SCHEMA
schema = (StructType([
    StructField('nm_cliente', StringType(), True),
    StructField('data', StringType(), True),
    StructField('pcd_cliente', StringType(),True),
    StructField('pzo', StringType(),True)
]))

In [ ]:
# Criando a coluna dt_carga para obter a data atual
df = df.withColumn('dt_carga', F.date_formtat(F.current_timestamp(), 'dd-MM-yyyy'))

In [ ]:
#Convertendo os timpos primitivos da coluna data_d
df = df.withColumn('data_d', F.col('data_d').cast(DateType()))

In [ ]:
# transformando a coluna em uma lista para fazer o calculo da funcao date_add
list_pzo = df.select('pzo').rdd.flatmap(lambda x:x).collect()

In [ ]:
#registrando a funcao
def date_add_days(data_d, pzo):
    return data_d + timedelta(days=pzo)

In [ ]:
#Variavel registrar a funcao com UDF
date_add_function = F.udf(dateadd_days, DateType())

In [ ]:
# chamar a nossa UDF para fazer a soma de duas colunas do DF
df = df.withColumn('soma_data', date_add_function(df.data_d, df.pzo))

In [ ]:
# FAZENDO O JOIN ENTRE DOIS DATAFRAMES
df_final = df_2.join(df, df_2['cd_cliente'] == df['cd_cliente'], "inner").filter(F.col('soma_data') >= F.current_date())

In [ ]:
# Usando Expressoes SQL
df = df.withColumn("comprar", F.expr("CASE WHEN Ano < 2020 THEN 'sim' ELSE 'nao' END"))1 